In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ライブラリのインポート
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

In [ ]:
# ハイパーパラメータの定義、トークナイザの初期化
model_name = "jplu/tf-xlm-roberta-large"
max_length = 120
batch_size = 16

tokenizer = AutoTokenizer.from_pretrained(model_name)

# 学習後のモデルの重みの読み込み
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.load_weights("../input/watson-xlm-r-large-training/watson-tf-xlm-roberta-large.h5")

# テストデータの読み込み
test = pd.read_csv("../input/contradictory-my-dear-watson/test.csv")

In [ ]:
# 前提、仮説のテキストのみを抽出
test_text = test[['premise', 'hypothesis']].values.tolist()

# トークナイズ + インデキシング
test_encoded = tokenizer.batch_encode_plus(test_text, padding=True, max_length=120, truncation=True)

# TensorFlowのDatasetに変換
test_dataset = (
    tf.data.Dataset.from_tensor_slices((dict(test_encoded))).batch(batch_size)
)

# 推論
test_preds = model.predict(test_dataset)

# 推論したデータの確認。logitsがモデルの最終層の出力
test_preds

In [ ]:
# 提出用ファイルの作成
submission = pd.read_csv("../input/contradictory-my-dear-watson/sample_submission.csv")
submission['prediction'] = test_preds["logits"].argmax(axis=1)
submission.to_csv('submission.csv', index=False)
submission.head()